# Importing Modules

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,random_split,Subset
from torchvision import datasets, transforms, models # add models to the list
from torchvision.utils import make_grid
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

# Defining Transforms

In [14]:
transform = transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize((224,224)),             # resize shortest side to 224 pixels      
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
        ])

# Preparing the Data

In [15]:
# Set the path to the dataset directory
data_dir = "E:/Fiverr_Projects/dataset/dataset"

# Load the dataset from the directory
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Printing Dataset Classes 
dataset.classes

['bad', 'good']

In [16]:
# Split the dataset into training (80%) and testing (20%)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create data loaders for the training and testing sets
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Verify the data loading
for images, labels in train_loader:
    print(f"Batch of images shape: {images.shape}")
    print(f"Batch of labels shape: {labels.shape}")
    break

for images, labels in test_loader:
    print(f"Batch of images shape: {images.shape}")
    print(f"Batch of labels shape: {labels.shape}")
    break

Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels shape: torch.Size([32])
Batch of images shape: torch.Size([32, 3, 224, 224])
Batch of labels shape: torch.Size([32])


In [17]:
print('Lenght of train_data :',len(train_dataset))
print('Length of test_dataset',len(test_dataset))

Lenght of train_data : 58184
Length of test_dataset 14547


# Defining Model
Using PreTrained AlexNet

In [18]:
AlexNetmodel = models.alexnet(pretrained = True)
AlexNetmodel

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

####   Lets freeze the pretrained weight and biases so we don't back propegate through them.

In [19]:
for param in AlexNetmodel.parameters():
    param.requires_grad = False

#### Lets modify the ALEXNET classifier to our own needs.

In [20]:
AlexNetmodel.classifier = nn.Sequential(nn.Linear(9216,1024),
                                        nn.ReLU(),
                                        nn.Dropout(p=0.4),
                                        nn.Linear(1024,2),
                                        nn.LogSoftmax(dim=1))

In [21]:
AlexNetmodel

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, 

#### Definig Loss and Optimizer

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(AlexNetmodel.classifier.parameters(), lr=0.001)

### Training the Model on Pretrained AlexNet

In [19]:
import time
import torch

start_time = time.time()

epochs = 1

max_trn_batch = 800
max_tst_batch = 300
batch_size = 32  # Set this to the batch size you are using in your DataLoader

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        if b == max_trn_batch:
            break

        # Apply the model
        y_pred = AlexNetmodel(X_train)
        loss = criterion(y_pred, y_train)
 
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print interim results
        if (b+1) % 200 == 0:
            print(f'epoch: {i:2}  batch: {b+1:4} [{(b+1)*batch_size:6}/8000]  loss: {loss.item():10.8f}  '
                  f'accuracy: {trn_corr.item()*100/((b+1)*batch_size):7.3f}%')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            if b == max_tst_batch:
                break

            # Apply the model
            y_val = AlexNetmodel(X_test)

            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed


epoch:  0  batch:  200 [  6400/8000]  loss: 1.04957426  accuracy:  83.109%
epoch:  0  batch:  400 [ 12800/8000]  loss: 0.33428076  accuracy:  83.508%
epoch:  0  batch:  600 [ 19200/8000]  loss: 0.46128610  accuracy:  83.750%
epoch:  0  batch:  800 [ 25600/8000]  loss: 0.29655081  accuracy:  83.734%

Duration: 2626 seconds


Sorry for a little typo here total training images are around 58000 but here i have put 8000. I didn't want to stop in the middle of training make it correct but it dosen't effect the model accuracy.

#### Saving the Model

In [20]:
torch.save(AlexNetmodel.state_dict(), 'alexnet_model.pth')

### Let's Train it more

In [23]:
import time
import torch

start_time = time.time()

epochs = 1  # Change this to the number of additional epochs you want to train for


max_trn_batch = 1200
max_tst_batch = 450
batch_size = 32  # Set this to the batch size you are using in your DataLoader

train_losses = []
test_losses = []
train_correct = []
test_correct = []

# Load the previously trained model
AlexNetmodel.load_state_dict(torch.load('alexnet_model.pth'))
AlexNetmodel.train()  # Set the model to train mode

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        if b == max_trn_batch:
            break

        # Apply the model
        y_pred = AlexNetmodel(X_train)
        loss = criterion(y_pred, y_train)
 
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print interim results
        if (b+1) % 200 == 0:
            print(f'epoch: {i:2}  batch: {b+1:4} [{(b+1)*batch_size:6}/58000]  loss: {loss.item():10.8f}  '
                  f'accuracy: {trn_corr.item()*100/((b+1)*batch_size):7.3f}%')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            if b == max_tst_batch:
                break

            # Apply the model
            y_val = AlexNetmodel(X_test)

            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed


epoch:  0  batch:  200 [  6400/58000]  loss: 0.36588031  accuracy:  84.547%
epoch:  0  batch:  400 [ 12800/58000]  loss: 0.21997316  accuracy:  84.547%
epoch:  0  batch:  600 [ 19200/58000]  loss: 0.35470808  accuracy:  84.760%
epoch:  0  batch:  800 [ 25600/58000]  loss: 0.55783689  accuracy:  84.906%
epoch:  0  batch: 1000 [ 32000/58000]  loss: 0.43439680  accuracy:  84.737%
epoch:  0  batch: 1200 [ 38400/58000]  loss: 0.27563524  accuracy:  84.641%

Duration: 2241 seconds


In [24]:
torch.save(AlexNetmodel.state_dict(), 'alexnet_model.pth')

#### Loading the PreTrained Model

In [25]:
AlexNetmodel.load_state_dict(torch.load('alexnet_model.pth'))
AlexNetmodel.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Linear(in_features=9216, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.4, 

#### Creating two directories to store good and bad images predicted by the model.

In [29]:
os.makedirs('good_images', exist_ok=True)
os.makedirs('bad_images', exist_ok=True)

#### Classifying and Saving Images

In [30]:
from torchvision import transforms
from PIL import Image
import shutil

# Define the transformation for the test images
test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Adjust to your model input size if different
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Path to the Testing folder
test_folder_path = 'Testing'

# List all images in the Testing folder
image_paths = [os.path.join(test_folder_path, img) for img in os.listdir(test_folder_path) if img.endswith(('png', 'jpg', 'jpeg'))]

# Loop through each image and classify it
for image_path in image_paths:
    # Open the image
    image = Image.open(image_path).convert('RGB')
    
    # Apply transformations
    image_tensor = test_transforms(image).unsqueeze(0)  # Add batch dimension
    
    # Run the model on the image
    with torch.no_grad():
        output = AlexNetmodel(image_tensor)
        _, predicted = torch.max(output, 1)
        
    # Get the predicted class (assuming 0 for bad and 1 for good)
    class_name = 'good' if predicted.item() == 1 else 'bad'
    
    # Define the target folder
    target_folder = 'good_images' if class_name == 'good' else 'bad_images'
    
    # Save the image in the respective folder
    shutil.copy(image_path, os.path.join(target_folder, os.path.basename(image_path)))

print('Classification and saving completed.')


Classification and saving completed.
